In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import folium

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(link)
soup = BeautifulSoup(page.content,'html.parser')

table = soup.find('table')
table_rows = table.find_all('tr')
rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text[:-1] for i in td]
    if(len(row)>0 and row[1]!='Not assigned'):
        s ='' 
        for i in range(2,len(row)):
            s = s + row[i]
        rows.append([row[0],row[1],s])
#         print([row[0],row[1],s])

df = pd.DataFrame(rows,columns = ['Postal Code','Borough','Neighborhood'])

In [3]:
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df.shape

(103, 3)

In [5]:
def getXY(city):
    geolocator = Nominatim(user_agent="ca_explorer")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    return [latitude,longitude]

In [6]:
areas = df['Borough'].unique()
areas

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [ ]:
# xy = {}
# for i in areas:
#     print(i)
#     t = getXY("{} Ontario, Canada".format(i))
#     xy[i] = t
# # print(xy)
# getXY('M5A Canada')
# geolocator = Nominatim(user_agent="ca_explorer")
# print(geolocator.geocode({"postalcode": 'M3A'}))
# getXY('St. James Town Ontario, Canada')

In [7]:
xs = []
ys = []

for i in range(0,103):
    a = '{} Ontario, Canada'.format(df.iloc[i,2].split(',')[0])
#     print(i,a)
    try:
        t = getXY(a)  
        xs.append(t[0])
        ys.append(t[1])
    except:
        xs.append(None)
        ys.append(None)
df['Latitude'] = xs
df['Longitude'] = ys

In [8]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.659659,-79.390340


In [21]:
print(getXY(' Toronto Ontario, Canada'))
map_newyork = folium.Map(location=getXY(' Toronto Ontario, Canada'), zoom_start=10)
df1 = df[df['Latitude']!=None]
# df1['Latitude'] = df1['Latitude'].astype('float')
# if(None in df1['Longitude']):
#     print("Yes")
# else:
#     print('No')
# df1['Longitude'] = df1['Longitude'].astype('float')
# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
#     print(lat,lng)
    if(lat==lat and lng==lng):
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

[43.6534817, -79.3839347]
